In [1]:
import pandas as pd
import glob
import os
import plotly.express as px
import plotly.graph_objects as go



import pprint
pp = pprint.PrettyPrinter()

In [8]:
# Load and display datasets 
path = r'/Users/shakeel/Documents/paklytics/economy'
allFiles = glob.glob(path + "/*.csv")
allFiles

# Read in the data + save as a dict
dataStorage_economy = {}
for filename in allFiles:
    name = os.path.basename(filename).split(".")[0]
    #print(name)
    dataStorage_economy[name] = pd.read_csv(filename, skiprows=16, na_values='NaN')

print("Economy - Available Datasets:")
print("-----------------------------")
pp.pprint(list(dataStorage_economy.keys()))

Economy - Available Datasets:
-----------------------------
['pakistan-inflation-rate-cpi',
 'pakistan-gni-gross-national-income',
 'pakistan-economic-growth-rate',
 'pakistan-gdp-gross-domestic-product',
 'pakistan-gdp-growth-rate',
 'preprocessed',
 'pakistan-gdp-per-capita',
 'pakistan-gni-per-capita',
 'pakistan-debt-to-gdp-ratio',
 'pakistan-gnp-gross-national-product',
 'pakistan-manufacturing-output']


In [9]:
# Load and display datasets 
path = r'/Users/shakeel/Documents/paklytics/trade'
allFiles = glob.glob(path + "/*.csv")
allFiles

# Read in the data + save as a dict
dataStorage_trade = {}
for filename in allFiles:
    name = os.path.basename(filename).split(".")[0]
    #print(name)
    dataStorage_trade[name] = pd.read_csv(filename, skiprows=16, na_values='NaN')

print("Trade - Available Datasets:")
print("---------------------------")
pp.pprint(list(dataStorage_trade.keys()))

Trade - Available Datasets:
---------------------------
['pakistan-tariff-rates',
 'pakistan-foreign-direct-investment',
 'pakistan-exports',
 'pakistan-tourism-statistics',
 'pakistan-imports',
 'pakistan-trade-gdp-ratio',
 'pakistan-trade-balance-deficit']


In [16]:
pex = pd.read_csv('files/pex.csv')

pex = pex.T
pex = pex[1:-1]
pex = pex.reset_index(drop=True)
pex

,0
0,4.7619
1,4.7619
2,4.7619
3,4.7619
4,4.7619
...,...
57,105.455162
58,121.824069
59,150.036254
60,161.83848


In [17]:
gdp = dataStorage_economy['pakistan-gdp-gross-domestic-product']
imp = dataStorage_trade['pakistan-imports']
exp = dataStorage_trade['pakistan-exports']

print(gdp)
print(imp)
print(exp)

          date   GDP ( Billions of US $)   Per Capita (US $)   Annual % Change
0   1960-12-31                  3.749265             83.3379               NaN
1   1961-12-31                  4.118648             89.4090            5.9873
2   1962-12-31                  4.310164             91.3192            4.4829
3   1963-12-31                  4.630827             95.7034            8.6888
4   1964-12-31                  5.204956            104.8802            7.5698
..         ...                       ...                 ...               ...
57  2017-12-31                339.205616           1631.5319            4.4326
58  2018-12-31                356.128225           1678.0432            6.1517
59  2019-12-31                320.909489           1481.8139            2.4976
60  2020-12-31                300.306332           1359.5145           -1.3295
61  2021-12-31                346.343170           1537.9364            6.0298

[62 rows x 4 columns]
          date   Billions of 

In [ ]:
gdp = gdp.rename(columns = {'date': 'year', ' GDP ( Billions of US $)': 'GDP_USD_billion', ' Per Capita (US $)': 'GDP_perCapita_USD', ' Annual % Change': 'GDP_percentage_change' }, inplace = False)
gdp['year'] = pd.DatetimeIndex(gdp['year']).year
gdp

In [ ]:
exp.columns

In [ ]:
imp = imp.rename(columns = {'date': 'year', ' Billions of US $': 'imports_USD_billion', ' % of GDP': 'import_GDP_percentage'}, inplace = False)
exp = exp.rename(columns = {'date': 'year', ' Billions of US $': 'exports_USD_billion', ' % of GDP': 'export_GDP_percentage'}, inplace = False)

imp['year'] = pd.DatetimeIndex(imp['year']).year
exp['year'] = pd.DatetimeIndex(exp['year']).year

print(imp)
print(exp)

In [ ]:
data = pd.merge(gdp, imp, on='year', how='outer')
data = pd.merge(data, exp, on='year', how='outer')
del data[' _x']
del data[' _y']
del data['GDP_percentage_change']
data

In [ ]:
data = pd.merge (data, tl, on='year', how='outer')
data

In [ ]:
# data.dtypes
data = data[data['year'] >= 1960].reset_index()
del data['index']
data
                 

In [ ]:
data.columns
data = data[['year', 'imports_USD_billion', 'exports_USD_billion']]
data

In [ ]:
df = (data
    .reset_index()
    .melt(id_vars='year', var_name='stats', value_name='value')
    .set_index('year')
)
display(df.to_string())


In [ ]:
df = df[df.stats != 'index']

#df['year'] = df.index
df.reset_index(inplace=True)
df

In [ ]:
imports=df[df['stats'].isin(['imports_USD_billion'])]
exports=df[df['stats'].isin(['exports_USD_billion'])]

imports

In [ ]:
trace1 = go.Scatter(x=imports['year'][:2],
                    y=imports['value'][:2],
                    mode='lines',
                    line=dict(width=1.5))


trace2 = go.Scatter(x = exports['year'][:2],
                    y = exports['value'][:2],
                    mode='lines',
                    line=dict(width=1.5))


frames = [dict(data= [dict(type='scatter',
                           x=imports['year'][:k+1],
                           y=imports['value'][:k+1]),
                      dict(type='scatter',
                           x=exports['year'][:k+1],
                           y=exports['value'][:k+1]),
                     ],
               traces= [0, 1],  
              )for k  in  range(1, len(imports)-1)]

layout = go.Layout(width=1000,
                   height=600,
                   showlegend=False,
                   hovermode='x unified',
                   updatemenus=[
                        dict(
                            type='buttons', showactive=False,
                            y=1.05,
                            x=1.15,
                            xanchor='right',
                            yanchor='top',
                            pad=dict(t=0, r=10),
                            buttons=[dict(label='Play',
                            method='animate',
                            args=[None, 
                                  dict(frame=dict(duration=100, 
                                                  redraw=False),
                                                  transition=dict(duration=10),
                                                  fromcurrent=True,
                                                  mode='immediate')]
                            )]
                        ),
                        dict(
                            type = "buttons",
                            direction = "left",
                            buttons=list([
                                dict(
                                    args=[{"yaxis.type": "linear"}],
                                    label="LINEAR",
                                    method="relayout"
                                ),
                                dict(
                                    args=[{"yaxis.type": "log"}],
                                    label="LOG",
                                    method="relayout"
                                )
                            ]),
                        ),
                    ]              
                  )

layout.update(xaxis =dict(range=['1960', '2022'], autorange=False),
              yaxis =dict(range=[0, 80], autorange=False));



fig = go.Figure(data=[trace1, trace2], frames=frames, layout=layout)


fig.show()


In [ ]:
fig = px.line(df, x=df.index, y="value", color="stats", 
             custom_data=['stats', 'value' ])


fig.update_traces(mode="markers+lines")

fig.update_traces(
    hovertemplate="<br>".join([
        #"Year: %{x}",
        #"ColY: %{y}",
        #"%{customdata[0]}",
        "Value: %{customdata[1]}",
        #"Col3: %{customdata[2]}",
    ])
)

fig.update_layout(
    #hovermode='x unified',
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=[{"yaxis.type": "linear"}],
                    label="LINEAR",
                    method="relayout"
                ),
                dict(
                    args=[{"yaxis.type": "log"}],
                    label="LOG",
                    method="relayout"
                )
            ]),
        ),
    ]
)

# fig.add_vrect(x0="2018", x1="2021", 
#               annotation_text="PTI", annotation_position="top left",
#               fillcolor="green", opacity=0.25, line_width=0)

fig.show()

In [ ]:
df = gdp
fig = px.line(gdp, x="date", y=" GDP ( Billions of US $)", title='Pakistan GDP')
fig.add_scatter(x=imp['date'], y=imp[' Billions of US $'], mode='lines')
fig.add_scatter(x=exp['date'], y=exp[' Billions of US $'], mode='lines')



fig.show()